<center><img src="../Picture Data/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# Autopilot - train model

Train a neural network to get an input image and output a set of x, y values corresponding to a target.
We will use the PyTorch deep learning framework we used in the previous course to train the ResNet18 neural network structure model to identify road conditions for automatic driving.

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

### Download and Get data
> # If you are training on JetBot, you can skip this step!

In [ ]:
!unzip -q road_following.zip

After unzipping, you will see a folder named ' 'dataset_all ' ' appear in the file browser.

### Create a database instance


We create a custom 'torch.utils.data.Dataset` database instance that implements the ``__len__`` and ``__getitem__`` functions.


This class is responsible for loading the image and parsing the x and y values in the image file name.
Since we implemented the ``torch.utils.data.Dataset`` class, we can use all of the torch data utilities, and we hardcoded some conversions (such as color jitter) in the dataset. 

We set the random horizontal flip to optional (if you want to follow an asymmetrical path, such as a road), it doesn't matter if Jetbot follows a certain convention, you can enable flips to augment the dataset.

In [ ]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
dataset = XYDataset('dataset_xy', random_hflips=False)

### Split the data set into a training sets and a test sets 

Once we read the dataset, we will split the dataset in the training set and test set. In this example, we split the training and tested 90%-10%. The test set will be used to verify the accuracy of the model we are training.

In [ ]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create a data loader to load data in bulk

We use the ``DataLoader`` class to load data in bulk, shuffle data, and allow multiple child processes to be used.

In this example, we use a data batch size of 64. The batch size will be based on the memory available to the GPU, which can affect the accuracy of the model. 

In [ ]:
# Train sets
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

# Test sets
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define a neural network model

The ResNet-18 model we use is based on PyTorch TorchVision. In the process of “migration learning”, also called “transfer learning”, we can reuse a pre-trained model (training millions of images) for one possible A new task with much less data available. 

For more information, please visit ResNet-18:
Https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

More details about transfer learning (requires science online):
https://www.youtube.com/watch?v=yofjFQddwHE

In [ ]:
model = models.resnet18(pretrained=True)

The ResNet model is fully connected (fc) to the final layer with 512 as ``in_features``, we will train the regression, so ``out_features`` as 1

Finally, we transfer the model to the GPU for execution.

In [ ]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

# Train Regression

We have trained 50 times. If there is a loss reduction situation, we will save the best model:

In [ ]:
# NUM_EPOCHS = 70
NUM_EPOCHS = 50
BEST_MODEL_PATH = 'best_steering_model_xy.pth'
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('%f, %f' % (train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss

Once the model is trained, it will generate a ``best_steering_model_xy.pth`` file, which we will use in the autopilot routine for reasoning.